# Topic Visualization with pyLDAvis

In [1]:
import pandas as pd
import random

import string 
import re

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis.sklearn


In [6]:
pyLDAvis.enable_notebook()

eng_stopwords_and_punct = stopwords.words('english') + list(string.punctuation) + ['’', '“','”', "''", '""', '', '..', '...', '``', '—', '->', '’ ’']

# TweetTokenizer will put all text in the tweet to lowercase, strip out usernames/handles and reduce reated chars in words
tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True) 

lemmatizer = WordNetLemmatizer()

# Functions
def clean_tokenize_lemmatize_tweet(tweet):
    ''' 
    Apply text cleaning, tokenization then lemmatization on the given tweet 
    '''
    #remove urls
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)
    #remove RT 
    tweet = re.sub(r'^RT\s+', '', tweet)
    #remove the # symbol
    tweet = re.sub('#', '', tweet)
    #remove words containing numbers
    tweet = re.sub(r'\w*\d\w*', '', tweet)
    #tokenize
    tokens = tweet_tokenizer.tokenize(tweet)
    # remove stopwords and punctuation
    tokens = [token for token in tokens if token not in eng_stopwords_and_punct]
    # remove tokens that are only 1 char in length
    tokens = [token for token in tokens if len(token)>1]
    #lemmatize
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens


def perform_topic_modeling(content, topic_num, top_word_count):
    '''
    Perform topic modeling using on the provided corpus for the 
    designated number of topics and top word count
    '''
    cv = CountVectorizer(max_df=0.95, 
                         min_df=0.01, 
                         strip_accents='unicode', 
                         tokenizer=clean_tokenize_lemmatize_tweet, 
                         ngram_range=(1,2))

    lda = LatentDirichletAllocation(max_iter=10, 
                                    learning_method='online', 
                                    n_components=topic_num, 
                                    random_state=42)

    # Unsupervised learning...no need to do a test/train split here ;)
    dtm = cv.fit_transform(content)

    # This might take awhile if you are dealing with a large amount of documents!
    lda.fit(dtm)

    return (lda, dtm, cv)



In [9]:
# Read in all US Tweets and create Series for entire US and Regions of interest 
vetted_usa_tweets = pd.read_csv('../data/vetted_usa_tweets.csv')
us_tweets = vetted_usa_tweets['content']
west_tweets = vetted_usa_tweets[vetted_usa_tweets['region'] == 'West']['content']
midwest_tweets = vetted_usa_tweets[vetted_usa_tweets['region'] == 'Midwest']['content']
southwest_tweets = vetted_usa_tweets[vetted_usa_tweets['region'] == 'Southwest']['content']
southeast_tweets = vetted_usa_tweets[vetted_usa_tweets['region'] == 'Southeast']['content']
northeast_tweets = vetted_usa_tweets[vetted_usa_tweets['region'] == 'Northeast']['content']


## US Topics

In [12]:
# model topics in all the Tweets
usa_tm_tuple = perform_topic_modeling(us_tweets, 15, 30)
pyLDAvis.sklearn.prepare(usa_tm_tuple[0], usa_tm_tuple[1], usa_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.121989  0.243051       1        1  11.915581
3      0.140570  0.172117       2        1  11.057243
4     -0.074935  0.172123       3        1   7.790033
14    -0.080651 -0.267210       4        1   7.646260
6      0.154334  0.145617       5        1   7.468707
1      0.251925 -0.044270       6        1   7.408205
13     0.278624 -0.065378       7        1   7.202922
10    -0.106469  0.019062       8        1   6.148228
7     -0.109208  0.124135       9        1   5.671316
0      0.285099 -0.083863      10        1   5.577889
11    -0.161609 -0.022262      11        1   5.573480
5     -0.126836 -0.053065      12        1   5.444338
2     -0.118224  0.018050      13        1   4.538045
9     -0.121225 -0.093685      14        1   3.765880
12    -0.089406 -0.264424      15        1   2.791871, topic_info=                  Term          Freq         Total Category  logprob  loglift
44               class   7515.000000   7515.000000  Default  30.0000  30.0000
196             online  18878.000000  18878.000000  Default  29.0000  29.0000
55              course   5580.000000   5580.000000  Default  28.0000  28.0000
293           teaching   5452.000000   5452.000000  Default  27.0000  27.0000
256           semester   3438.000000   3438.000000  Default  26.0000  26.0000
..                 ...           ...           ...      ...      ...      ...
13   assignment online      0.081103    667.048110  Topic15 -11.6399  -5.4364
56     course research      0.081103    565.342208  Topic15 -11.6399  -5.2710
70                  dm      0.081103   1528.431189  Topic15 -11.6399  -6.2656
298               term      0.081103    341.439223  Topic15 -11.6399  -4.7667
210              paper      0.081103   2107.611774  Topic15 -11.6399  -6.5869

[548 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0        12  0.997048        able
1         5  0.999121      access
2        13  0.996910    activity
3         4  0.998079  affordable
4         4  0.999233     algebra
...     ...       ...         ...
337       7  0.997452     writing
338       1  0.330883        year
338       2  0.000789        year
338       8  0.668075        year
339      14  0.998605        zoom

[467 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 4, 5, 15, 7, 2, 14, 11, 8, 1, 12, 6, 3, 10, 13])

## West Region Topics

In [13]:
# model topics in all West Region Tweets
w_tm_tuple = perform_topic_modeling(west_tweets, 10, 30)
pyLDAvis.sklearn.prepare(w_tm_tuple[0], w_tm_tuple[1], w_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.159059  0.129637       1        1  18.346011
8     -0.331049  0.162249       2        1  13.613608
0      0.124504 -0.190637       3        1  10.881974
5      0.066042  0.136283       4        1  10.485657
2      0.031306 -0.138518       5        1  10.358785
3      0.179604 -0.093950       6        1   9.708269
9     -0.130864  0.027520       7        1   8.357031
6      0.041426 -0.152466       8        1   8.330499
7     -0.293510 -0.135472       9        1   5.090197
1      0.153483  0.255353      10        1   4.827968, topic_info=                  Term         Freq        Total Category  logprob  loglift
178             online  3171.000000  3171.000000  Default  30.0000  30.0000
142           learning  2493.000000  2493.000000  Default  29.0000  29.0000
223             school  1595.000000  1595.000000  Default  28.0000  28.0000
61            distance  1547.000000  1547.000000  Default  27.0000  27.0000
62   distance learning  1396.000000  1396.000000  Default  26.0000  26.0000
..                 ...          ...          ...      ...      ...      ...
237             social     0.128173   113.508227  Topic10 -10.1331  -3.7555
172                new     0.140028   489.356707  Topic10 -10.0447  -5.1283
152            looking     0.128150   150.850254  Topic10 -10.1333  -4.0401
214             remote     0.127399   137.808097  Topic10 -10.1392  -3.9555
49              course     0.128005   999.555073  Topic10 -10.1344  -5.9323

[401 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         7  0.988019     able
1         2  0.112959  academy
1         4  0.883738  academy
2         4  0.369260   access
2         6  0.620671   access
...     ...       ...      ...
311       1  0.001672     year
311       3  0.996636     year
312       1  0.041468     zoom
312       5  0.347298     zoom
312       9  0.606475     zoom

[576 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 9, 1, 6, 3, 4, 10, 7, 8, 2])

## Southwest Region Topics

In [14]:
# model topics in all Southwest Region Tweets
sw_tm_tuple = perform_topic_modeling(southwest_tweets, 10, 30)
pyLDAvis.sklearn.prepare(sw_tm_tuple[0], sw_tm_tuple[1], sw_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.212719 -0.085072       1        1  12.858680
9     -0.229172 -0.090313       2        1  11.731128
2     -0.152103 -0.073701       3        1  11.173768
1      0.161898 -0.003870       4        1  10.336374
8      0.247565 -0.053994       5        1  10.325890
7     -0.087780  0.060638       6        1  10.063241
0     -0.246763 -0.047932       7        1   9.366781
3     -0.064997 -0.062966       8        1   8.745886
4      0.152701 -0.085404       9        1   8.464116
6      0.005933  0.442613      10        1   6.934136, topic_info=             Term         Freq        Total Category  logprob  loglift
172        online  1822.000000  1822.000000  Default  30.0000  30.0000
253      teaching   559.000000   559.000000  Default  29.0000  29.0000
36          class   679.000000   679.000000  Default  28.0000  28.0000
45         course   507.000000   507.000000  Default  27.0000  27.0000
252       teacher   405.000000   405.000000  Default  26.0000  26.0000
..            ...          ...          ...      ...      ...      ...
249          take    78.385990   167.465258  Topic10  -3.2724   1.9096
124   instruction    42.409598    88.565419  Topic10  -3.8867   1.9323
107          help    58.527389   271.030244  Topic10  -3.5646   1.1360
173  online class    50.540260   169.852083  Topic10  -3.7113   1.4566
164          need    45.006964   253.394323  Topic10  -3.8272   0.9406

[412 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         9  0.976484     able
1         1  0.487122   access
1         6  0.334897   access
1         7  0.167448   access
2         8  0.985549  account
...     ...       ...      ...
302       4  0.024400     year
303       2  0.441664     zoom
303       4  0.220832     zoom
303       5  0.237819     zoom
303       8  0.084935     zoom

[651 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 10, 3, 2, 9, 8, 1, 4, 5, 7])

## Midwest Region Topics

In [15]:
# model topics in all Midwest Region Tweets
mw_tm_tuple = perform_topic_modeling(midwest_tweets, 10, 30)
pyLDAvis.sklearn.prepare(mw_tm_tuple[0], mw_tm_tuple[1], mw_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.067018 -0.055804       1        1  14.677201
0     -0.292383 -0.098323       2        1  12.750364
1      0.046592  0.242969       3        1  12.566665
6     -0.267113 -0.088487       4        1  11.888346
2      0.099416  0.006434       5        1  10.976515
9      0.172610 -0.186035       6        1   8.407473
7      0.149397 -0.229195       7        1   7.928469
3      0.110103  0.146228       8        1   7.750475
4      0.127557  0.014152       9        1   7.659385
8     -0.079160  0.248063      10        1   5.395108, topic_info=         Term         Freq        Total Category  logprob  loglift
167    online  2415.000000  2415.000000  Default  30.0000  30.0000
33      class   867.000000   867.000000  Default  29.0000  29.0000
251  teaching   755.000000   755.000000  Default  28.0000  28.0000
43     course   712.000000   712.000000  Default  27.0000  27.0000
218    school  1010.000000  1010.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
195   project    10.961572    80.977047  Topic10  -5.3215   0.9199
33      class   107.430813   867.888110  Topic10  -3.0391   0.8305
218    school    91.950236  1010.482617  Topic10  -3.1947   0.5227
100      help    28.545019   299.854497  Topic10  -4.3645   0.5679
167    online    15.022843  2415.397470  Topic10  -5.0064  -2.1604

[400 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.017412     able
0        10  0.975067     able
1         6  0.147040  academy
1         8  0.840230  academy
2         7  0.246839   access
...     ...       ...      ...
295       8  0.497412     year
295       9  0.004829     year
296       1  0.214855     zoom
296       4  0.300796     zoom
296      10  0.472680     zoom

[614 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 2, 7, 3, 10, 8, 4, 5, 9])

## Northeast Region Topics

In [16]:
# model topics in all Northeast Region Tweets
ne_tm_tuple = perform_topic_modeling(northeast_tweets, 10, 30)
pyLDAvis.sklearn.prepare(ne_tm_tuple[0], ne_tm_tuple[1], ne_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.145680 -0.048198       1        1  19.077333
6     -0.127910 -0.013867       2        1  12.562864
8     -0.229098 -0.000658       3        1  12.408881
5     -0.197601  0.060119       4        1  10.609068
3      0.045215  0.251619       5        1   9.396787
0     -0.240988 -0.024932       6        1   8.627531
9      0.138635 -0.167802       7        1   8.528037
2      0.239591 -0.108872       8        1   7.483361
1      0.191940  0.299372       9        1   6.196127
4      0.034534 -0.246781      10        1   5.110011, topic_info=         Term         Freq        Total Category  logprob  loglift
33      class   955.000000   955.000000  Default  30.0000  30.0000
173    online  3096.000000  3096.000000  Default  29.0000  29.0000
140  learning  1369.000000  1369.000000  Default  28.0000  28.0000
44     course   960.000000   960.000000  Default  27.0000  27.0000
263  teaching   945.000000   945.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
100        go    30.287660   183.641422  Topic10  -4.4252   1.1717
173    online   230.712375  3096.138193  Topic10  -2.3947   0.3772
109      help    42.812652   372.291666  Topic10  -4.0791   0.8111
252   student    37.706841  1112.329479  Topic10  -4.2061  -0.4104
306      year    22.776153   311.171009  Topic10  -4.7102   0.3593

[403 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         3  0.692751    access
0         4  0.085408    access
0         7  0.218264    access
1         3  0.141178  activity
1         4  0.023530  activity
...     ...       ...       ...
305       1  0.339018     would
305       4  0.657279     would
306       2  0.925536      year
306      10  0.073914      year
307       1  0.992118      zoom

[593 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 9, 6, 4, 1, 10, 3, 2, 5])

## Southeast Region Topics

In [17]:
# model topics in all Southeast Region Tweets
se_tm_tuple = perform_topic_modeling(southeast_tweets, 10, 30)
pyLDAvis.sklearn.prepare(se_tm_tuple[0], se_tm_tuple[1], se_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.361170 -0.001947       1        1  18.367855
2      0.027108  0.175495       2        1  12.137625
0      0.036360 -0.111623       3        1  10.123179
5     -0.284202 -0.177796       4        1  10.060666
4      0.164357 -0.147951       5        1   9.791471
1      0.114037  0.008439       6        1   9.767253
9      0.131989 -0.122375       7        1   9.261319
6      0.128226 -0.158633       8        1   8.709958
3     -0.036412  0.300027       9        1   6.566409
8      0.079707  0.236364      10        1   5.214265, topic_info=          Term         Freq        Total Category  logprob  loglift
168     online  2703.000000  2703.000000  Default  30.0000  30.0000
254   teaching   924.000000   924.000000  Default  29.0000  29.0000
34       class   957.000000   957.000000  Default  28.0000  28.0000
244    student  1103.000000  1103.000000  Default  27.0000  27.0000
217     school  1014.000000  1014.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
73     excited    30.595847   151.381010  Topic10  -4.3750   1.3548
200        put     9.721615    50.149166  Topic10  -5.5216   1.3131
244    student   166.989921  1103.970360  Topic10  -2.6780   1.0650
266      today    44.091634   309.738760  Topic10  -4.0096   1.0043
61   education    20.132917   539.753172  Topic10  -4.7935  -0.3350

[396 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         2  0.241348     able
0         7  0.120674     able
0         9  0.618455     able
1         1  0.032543   access
1         3  0.423057   access
...     ...       ...      ...
294       6  0.086780  working
295       3  0.981164    world
296       9  0.995638    would
297       5  0.997157     year
298       2  0.991944     zoom

[619 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 3, 1, 6, 5, 2, 10, 7, 4, 9])